In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras import utils
from rapidae.data.datasets import load_MNIST
from rapidae.data.utils import evaluate, display_diff
from rapidae.models.base.default_architectures import (VanillaEncoder, VanillaDecoder)
from rapidae.models.cae.cae_model import CAE
from rapidae.pipelines.training import TrainingPipeline

2023-12-25 13:17:49.812797: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 13:17:49.812827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 13:17:49.813477: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-25 13:17:49.818155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-25 13:17:50.439207: W tensorflow/compiler/tf2

In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-25 13:17:51 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-25 13:17:51 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-25 13:17:51 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-25 13:17:51 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = CAE(input_dim=x_train.shape[1], latent_dim=2,
            encoder=VanillaEncoder, decoder=VanillaDecoder, layers_conf=[32, 64])

2023-12-25 13:17:51.456467: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-25 13:17:51.473568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-25 13:17:51.473809: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
x_train.shape

(60000, 784)

In [6]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=10)

trained_model = pipe(x=x_train, y=y_train)

2023-12-25 13:17:51 [INFO]: +++ training_pipeline +++
2023-12-25 13:17:51 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-25_13-17-51


Epoch 1/10

Epoch 1: loss improved from inf to 7269.98682, saving model to ../output_dir/training_pipeline_2023-12-25_13-17-51/model.weights.h5
469/469 - 18s - 37ms/step - contractive_loss: 46.0038 - loss: 7269.9868 - reconstruction_loss: 7223.6162
Epoch 2/10

Epoch 2: loss improved from 7269.98682 to 7217.07568, saving model to ../output_dir/training_pipeline_2023-12-25_13-17-51/model.weights.h5
469/469 - 17s - 36ms/step - contractive_loss: 2.9762e-04 - loss: 7217.0757 - reconstruction_loss: 7217.0894
Epoch 3/10

Epoch 3: loss improved from 7217.07568 to 7217.04834, saving model to ../output_dir/training_pipeline_2023-12-25_13-17-51/model.weights.h5
469/469 - 16s - 35ms/step - contractive_loss: 1.5224e-04 - loss: 7217.0483 - reconstruction_loss: 7217.0645
Epoch 4/10

Epoch 4: loss did not improve from 7217.04834
469/469 - 16s - 35ms/step - contractive_loss: 5.5808e-05 - loss: 7217.0513 - reconstruction_loss: 7217.0718
Epoch 5/10

Epoch 5: loss improved from 7217.04834 to 7217.04736, s

2023-12-25 13:19:31.196958: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 392.0KiB (rounded to 401408)requested by op RealDiv
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-12-25 13:19:31.197044: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-12-25 13:19:31.197057: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 76, Chunks in use: 73. 19.0KiB allocated for chunks. 18.2KiB in use in bin. 3.3KiB client-requested in use in bin.
2023-12-25 13:19:31.197065: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 11, Chunks in use: 8. 6.2KiB allocated for chunks. 4.5KiB in use in bin. 4.0KiB client-requested in use in bin.
2023-12-25 13:19:31.197073: I

ResourceExhaustedError: {{function_node __wrapped__RealDiv_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:RealDiv] name: 

In [ ]:
y_hat = trained_model.predict(x_test)

In [ ]:
print(x_test.shape)
print(y_hat['recon'].shape)
display_diff(x_test, y_hat['recon'])